In [1]:
# -*- coding: utf-8 -*-

import pickle
import numpy as np
import tensorflow as tf
import collections
from tqdm import tqdm

with open('wiki.zh.word.text', 'rb') as fr:
    lines = fr.readlines()
print('共%d行' % len(lines))
print(lines[0].decode('utf-8'))

共254419行
欧几里得 西元前 三 世纪 的 希腊 数学家 现在 被 认为 是 几何 之 父 此画 为 拉斐尔 的 作品 雅典 学院 数学 mathematics 是 利用 符号语言 研究 数量 结构 变化 以及 空间 等 概念 的 一门 学科 从 某种 角度看 属于 形式 科学 的 一种 数学 透过 抽象化 和 逻辑推理 的 使用 由 计数 计算 数学家 们 拓展 这些 概念 对 数学 基本概念 的 完善 早 在 古埃及 而 在 古希腊 那里 有 更为 严谨 的 处理 从 那时 开始 数学 的 发展 便 持续 不断 地 小幅 进展 世纪 的 文艺复兴 时期 致使 数学 的 加速 发展 直至 今日 今日 数学 使用 在 不同 的 领域 中 包括 科学 工程 医学 和 经济学 等 有时 亦 会 激起 新 的 数学 发现 并 导致 全新 学科 的 发展 数学家 也 研究 纯数学 就是 数学 本身 的 实质性 内容 而 不以 任何 实际 应用 为 目标 虽然 许多 研究 以 纯数学 开始 但 其 过程 中 也 发现 许多 应用 之 处 词源 西方 语言 中 数学 一 词源 自于 古希腊 语 的 其有 学习 学问 科学 数学 研究 即使 在 其 语源 内 其 形容词 意思 为 和 学习 有关 的 用功 的 亦 会 被 用来 指 数学 的 其 在 英语 中 表面 上 的 复数 形式 及 在 法语 中 的 表面 复数 形式 可溯 至 拉丁文 的 中性 复数 由 西塞罗 译自 希腊文 复数 此一 希腊语 被 亚里士多德 拿来 指 万物 皆 数 的 概念 数学 一词 的 大约 产生 于 宋元 时期 多指 象数 之学 但 有时 也 含有 今天 上 的 数学 意义 例如 秦九韶 的 数学 九章 永乐 大典 数书 九章 也 被 宋代 周密 所著 的 癸 辛杂识 记 为 数学 大略 数学 通轨 明代 柯尚 迁著 数学 钥 清代 杜知 耕著 数学 拾遗 清代 丁取忠 撰 直到 经过 中国 数学 名词 审查 委员会 研究 算学 数学 两词 的 使用 状况 后 确认 以 数学 表示 今天 意义 上 的 数学 含义 历史 奇普 印加帝国 时所 使用 的 计数 工具 数学 有着 久远 的 历史 中国 古代 的 六艺 之一 就 有 数学 一词 在 西方 有 希腊语 词源 mathematik ó 

In [2]:
lines = [line.decode('utf-8') for line in lines]
words = ' '.join(lines)
words = words.replace('\n', '').split(' ')
print('共%d个词' % len(words))

共148134974个词


In [3]:
# 词典长度为 50000
vocab_size = 50000
# 统计每个词出现的次数，并选出词频最高的 vocab_size - 1 个词
vocab = collections.Counter(words).most_common(vocab_size - 1)
# 向词典中添加 UNK 这个词
count = [['UNK', 0]]
count.extend(vocab)
print(count[:10])

[['UNK', 0], ('的', 5171536), ('在', 1437002), ('是', 1027031), ('为', 824117), ('年', 809836), ('和', 701575), ('了', 692395), ('小行星', 664887), ('与', 598554)]


In [4]:
# 词和 id 之间的相互映射
word2id = {}
id2word = {}
for i, w in enumerate(count):
    word2id[w[0]] = i
    id2word[i] = w[0]
print(id2word[100], word2id['数学'])

连结 1466


In [5]:
# 将语料转为 id 序列，⼀共有 22385926 个 UNK
data = []
for i in tqdm(range(len(lines))):
    line = lines[i].strip('\n').split(' ')
    d = []
    for word in line:
        if word in word2id:
            d.append(word2id[word])
        else:
            d.append(0)
            count[0][1] += 1
    data.append(d)
print('UNK数量%d' % count[0][1])

100%|██████████| 254419/254419 [01:55<00:00, 2207.45it/s]

UNK数量22385926


In [6]:
# 准备训练数据
X_train = []
Y_train = []
window = 3
for i in tqdm(range(len(data))):
    d = data[i]
    for j in range(len(d)):
        start = j - window
        end = j + window
        if start < 0:
            start = 0
        if end >= len(d):
            end = len(d) - 1
        while start <= end:
            # 跳过当前词本身
            if start == j:
                start += 1
                continue
            else:
                # 输入中添加的是当前词，输出添加的是 d[start]，所以我们使用的 Skip-Gram 模型
                X_train.append(d[j])
                Y_train.append(d[start])
                start += 1
# 从数组的形状中删除单维条目，即把 shape 中为 1 的维度去掉
X_train = np.squeeze(np.array(X_train))
Y_train = np.squeeze(np.array(Y_train))
# 扩展维数
Y_train = np.expand_dims(Y_train, -1)
print(X_train.shape, Y_train.shape)

100%|██████████| 254419/254419 [12:34<00:00, 337.07it/s]


(885756816,) (885756816, 1)


In [7]:
# 定义模型参数
batch_size = 128
embedding_size = 128
valid_size = 16
valid_range = 100
# 从 np.arange(valid_range) 中随机挑选 16 个数
valid_examples = np.random.choice(valid_range, valid_size, replace=False)
# 负采样的数量
num_negative_samples = 64

In [8]:
X = tf.placeholder(tf.int32, shape=[batch_size], name='X')
Y = tf.placeholder(tf.int32, shape=[batch_size, 1], name='Y')
valid = tf.placeholder(tf.int32, shape=[None], name='valid')
# 定义嵌入层并初始化
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, X)
# nce-loss
nce_weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocab_size]))
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, labels=Y, inputs=embed, num_sampled=num_negative_samples, num_classes=vocab_size))
# 使用 adam 优化器
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [9]:
# 将词向量标准化
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), axis=1, keep_dims=True))
normalized_embeddings = embeddings / norm
# 计算和给定词之间的相似度
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
# 创建 session
sess = tf.Session()
# 初始化所有变量
sess.run(tf.global_variables_initializer())
offset = 0
losses = []
for i in tqdm(range(1000000)):
    if offset + batch_size >= X_train.shape[0]:
        offset = (offset + batch_size) % X_train.shape[0]
    X_batch = X_train[offset: offset + batch_size]
    Y_batch = Y_train[offset: offset + batch_size]
    _, loss_ = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
    # 将 loss_ 添加到 losses 列表中
    losses.append(loss_)
    # 每 2000 步计算一下平均 loss 值并清空当前 losses 列表
    if i % 2000 == 0 and i > 0:
        print('Iteration %d Average Loss %f' % (i, np.mean(losses)))
        losses = []
    # 每 10000 步计算一下和给定词的相似度，并选出相似度最高的 5 个词
    if i % 10000 == 0:
        sim = sess.run(similarity, feed_dict={valid: valid_examples})
        for j in range(valid_size):
            valid_word = id2word[valid_examples[j]]
            top_k = 5
            nearests = (-sim[j, :]).argsort()[1: top_k + 1]
            s = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                s += ' ' + id2word[nearests[k]]
            print(s)  
    offset += batch_size

  0%|          | 1/1000000 [00:00<171:44:25,  1.62it/s]

Nearest to 其他: 侵蚀 鸭绿江 波利 官僚 gw
Nearest to 其中: 黄舒骏 hassan 皇室 第一份 纽卡斯尔
Nearest to 人: 致词 嘉兴市 二十大 usa 伟大
Nearest to 或: 舰长 邮政 光度 局 助阵
Nearest to 主要: 会议厅 universities 赫塔 芦 顺子
Nearest to 科罗: 戒备 盛宴 承接 兴起 歩
Nearest to 一个: aid 日落 解体 何方 新市镇
Nearest to and: 排序 得分 伊凡 morgan 城外
Nearest to 年: 国会 教区 一字 点 boyfriend
Nearest to 之: 工部尚书 娃娃 古建筑 争相 囊括
Nearest to 可以: 拾遗 野花 池锡辰 东湖 调动
Nearest to of: 更新 eat 从化 哪 国境
Nearest to 都: 减缓 肠胃 ha 备前 离异
Nearest to 中国: 高登 邢 政事 工序 初体验
Nearest to UNK: 诰 肥后 翌年 念 持节
Nearest to 美国: 国家队 利维坦 咸顿 龙大 实况


  0%|          | 2036/1000000 [00:11<1:33:45, 177.41it/s]

Iteration 2000 Average Loss 231.114838


  0%|          | 4022/1000000 [00:23<1:34:27, 175.73it/s]

...

100%|█████████▉| 996029/1000000 [1:32:20<00:21, 181.86it/s]

Iteration 996000 Average Loss 5.114169


100%|█████████▉| 998028/1000000 [1:32:31<00:11, 177.61it/s]

Iteration 998000 Average Loss 5.240217


100%|██████████| 1000000/1000000 [1:32:42<00:00, 179.78it/s]


In [11]:
# 保存模型
saver = tf.train.Saver()
saver.save(sess, './tf_128')

# 保存词向量，词和id相互映射用到的词典
final_embeddings = sess.run(normalized_embeddings)
with open('tf_128.pkl', 'wb') as fw:
    pickle.dump({'embeddings': final_embeddings, 'word2id': word2id, 'id2word': id2word}, fw, protocol=4)